<a href="https://colab.research.google.com/github/LucasMoreira7/FinanceAppDatabases/blob/main/Tratamento_base_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Conectando o notebook com o Drive
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import auth
import gspread

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Manipulação base de dados

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

pd.set_option('display.max_rows', 60)
pd.set_option('display.max_colum', 50)

# Carregar o DataFrame combinado de despesas e receitas
data_combined = pd.read_excel('/content/gdrive/MyDrive/FinanceDatabase/BaseReceitasDespesas_manipulada1.xlsx')
data_combined = data_combined.drop(columns=['usuario', 'descricao'])
data_combined = data_combined.sort_values(by='data')
data_combined = data_combined.rename(columns={'preço': 'preco'})

display(data_combined)


,preco,data,categoria,despesaOuReceita,parcela
73,19.50,2022-12-20,sem categoria,True,1
98,10.80,2022-12-20,sem categoria,True,1
97,9.90,2022-12-20,sem categoria,True,1
89,19.99,2022-12-20,casa,True,8
88,43.46,2022-12-20,Cachorro,True,1
...,...,...,...,...,...
501,25.78,2023-12-12,roupa,True,5
502,25.78,2024-01-11,roupa,True,4
503,25.78,2024-02-10,roupa,True,3
504,25.78,2024-03-11,roupa,True,2


In [ ]:
# Converter a coluna 'date' para o formato de data (datetime)

data_combined['data'] = pd.to_datetime(data_combined['data'], format='%d-%m-%Y', errors='coerce')

# Criar coluna de dia da semana em formato de string
data_combined["dia_da_semana"] = data_combined["data"].dt.strftime("%A")

# Criar coluna de dia da semana como número
data_combined["Dia_da_Semana"] = data_combined["data"].dt.weekday

data_combined['mes'] = data_combined['data'].dt.month

# Adicionar colunas dummy para o mês
data_combined = pd.concat([data_combined, pd.get_dummies(data_combined['data'].dt.month, prefix='Mes')], axis=1)

# Adicionar colunas dummy para o dia da semana
data_combined = pd.concat([data_combined, pd.get_dummies(data_combined['dia_da_semana'])], axis=1)

# Converter a coluna 'despesaOuReceita' para valores numéricos (1 para True e 0 para False)
data_combined['despesaOuReceita'] = data_combined['despesaOuReceita'].astype(int)

# Aplicar one-hot encoding nas colunas 'categoria'
data_encoded = pd.get_dummies(data_combined, columns=['categoria'], drop_first=True)

# Exibir o DataFrame resultante
print(data_encoded)

     preco       data  despesaOuReceita  parcela dia_da_semana  Dia_da_Semana  \
73   19.50 2022-12-20                 1        1       Tuesday              1   
98   10.80 2022-12-20                 1        1       Tuesday              1   
97    9.90 2022-12-20                 1        1       Tuesday              1   
89   19.99 2022-12-20                 1        8       Tuesday              1   
88   43.46 2022-12-20                 1        1       Tuesday              1   
..     ...        ...               ...      ...           ...            ...   
501  25.78 2023-12-12                 1        5       Tuesday              1   
502  25.78 2024-01-11                 1        4      Thursday              3   
503  25.78 2024-02-10                 1        3      Saturday              5   
504  25.78 2024-03-11                 1        2        Monday              0   
505  25.78 2024-04-10                 1        1     Wednesday              2   

     mes  Mes_1  Mes_2  Mes

In [ ]:
data_encoded.to_csv('/content/gdrive/MyDrive/FinanceDatabase/BaseCompletaTratada.csv', index=False)